# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

Описание данных:

* children — количество детей в семье
* days_employed — общий трудовой стаж в днях
* dob_years — возраст клиента в годах
* education — уровень образования клиента
* education_id — идентификатор уровня образования
* family_status — семейное положение
* family_status_id — идентификатор семейного положения
* gender — пол клиента
* income_type — тип занятости
* debt — имел ли задолженность по возврату кредитов
* total_income — ежемесячный доход
* purpose — цель получения кредита

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd
from pymystem3 import Mystem
from collections import Counter

Сохраним датасет в переменную df и посмотрим первые 10 строк датасета

In [2]:
df = pd.read_csv('/datasets/data.csv')

In [3]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Выведем информацию о датасете

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Посмотрим на количество пропусков

In [5]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Видно, что пропущенные значения находятся в столбцах `days_employed` и `total_income`, и их количество равно.

In [6]:
print('Доля пропущенных значений - {:.1%}'.format(df['days_employed'].isna().mean()))

Доля пропущенных значений - 10.1%


Посмотрим на сводную таблицу по всему датасету

In [7]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


**Вывод**

В датасете всего 21525 строк.  
1) Количество пропусков в столбцах `days_employed` и `total_income` совпадает и составляет около 10% от всего датасета. Это достаточно большое количество, необходимо разобраться как так получилось.  
2) В столбце `children` есть отрицательные значения, так же присутствую большие значения. Необходимо проверить эти значения, большие значения могут быть выбросами или опечаткой.  
3) Так же, на первый взгляд видно, что данные в столбце `days_employed` не соответствуют действительности, данные должны быть предоставлены в днях, а в стобце имеются как отрицательные, так и экстримально большие значения. Можно попробовать взять значения по модулю.  
4) В столбце `dob_years` минимальное значение равно 0. Скорее всего кто-то не верно указал свой возраст.     
5) В столбце `education` значения записана в разных регистрах, необходимо привести их к одному виду.  
6) Названия некоторых столбцов не отражают нужную информацию, необходимо их переименовать.

## Предобработка данных

### Изменение названий столбцов

Для лучшей информативности изменим название колонок. Для начала выведем текущие названия столбцов

In [8]:
df.columns.to_list()

['children',
 'days_employed',
 'dob_years',
 'education',
 'education_id',
 'family_status',
 'family_status_id',
 'gender',
 'income_type',
 'debt',
 'total_income',
 'purpose']

Зададим новые названия колонок

In [9]:
df.rename(columns={'children':'number_of_children', 
                   'dob_years':'client_age', 
                   'income_type':'employment_type', 
                   'debt':'credit_debt', 
                   'total_income':'monthly_income', 
                   'purpose':'credit_purpose'}, inplace=True)

Проверим что колонки переименовались

In [10]:
df.columns.to_list()

['number_of_children',
 'days_employed',
 'client_age',
 'education',
 'education_id',
 'family_status',
 'family_status_id',
 'gender',
 'employment_type',
 'credit_debt',
 'monthly_income',
 'credit_purpose']

### Обработка пропусков

Проверим, находятся ли в одних строках пропущенные значения в столбцах `days_employed` и `total_income`.

In [11]:
all(df[df['days_employed'].isna()].index == df[df['monthly_income'].isna()].index)

True

Пропущенные значения находятся в одних и тех же строках. Видимо тут есть зависимость, возможно были не указаны данные в одном из столбцов и это привило к ошибке в другом, либо это какая-то системаня ошибка. Попробуем разоабрться.  
Займемся столбцом `days_employed`.
Сделаем отрицательные значения положительными.

In [12]:
df['days_employed'] = df[df['days_employed'].notna()]['days_employed'].abs()

Чтобы проверить, что все получилось - выведем минимальное значение по столбцу `days_employed`. Оно должно быть положительным.

In [13]:
print(f"Минимальное значение в столбце 'days_employed' - {df['days_employed'].min()}")

Минимальное значение в столбце 'days_employed' - 24.14163324048118


Посмотрим на средние значение трудового стажа относительно типа занятости клиента.

In [14]:
df.pivot_table(index='employment_type', values='days_employed', aggfunc='mean')

,days_employed
employment_type,
безработный,366413.652744
в декрете,3296.759962
госслужащий,3399.896902
компаньон,2111.524398
пенсионер,365003.491245
предприниматель,520.848083
сотрудник,2326.499216
студент,578.751554


Из таблицы видно, что трудовой стаж по пенсионерам и безработным показывает экстримально высокие значения. Здесь явно допущена какая-то ошибка, возможно ошибка в указании отчетного года. Попробуем разобраться с этим дальше.

Посмотрим на сводную таблицу по типу занятости и возраста клиента. Выведем количество клиентов в группе, их минимальный, максимальный и средний возраст по группе.

In [15]:
df.pivot_table(index='employment_type', values='client_age', aggfunc=['count', 'min', 'max', 'mean'])

,count,min,max,mean
,client_age,client_age,client_age,client_age
employment_type,,,,
безработный,2,31,45,38.000000
в декрете,1,39,39,39.000000
госслужащий,1459,0,75,40.636737
компаньон,5085,0,74,39.697542
пенсионер,3856,0,74,59.063019
предприниматель,2,27,58,42.500000
сотрудник,11119,0,74,39.821027
студент,1,22,22,22.000000


Сразу бросается в глаза, что минимальный возраст в группах "госслужащий", "компаньон", "пенсионер" и "сотрудник" равны нулю, это явная ошибка. Предлагаю посмотреть на их количество в каждой группе.

In [16]:
df[df['client_age']==0].pivot_table(index='employment_type', values='client_age', aggfunc='count')

,client_age
employment_type,
госслужащий,6
компаньон,20
пенсионер,20
сотрудник,55


Клиентов в группах с возрастом равным нулю не очень много, предлагаю их заменить на среднее значение по каждой группе.

In [17]:
# сохраним название групп в список, далее пройдемся по каждой группе где возраст равен нулю и заменим в ней значение на среднее по группе
client_type = df[df['client_age']==0]['employment_type'].unique()

for type in client_type:
    df.loc[(df['client_age']==0) & (df['employment_type']==type), 'client_age'] = int(df[(df['client_age']!=0) & (df['employment_type']==type)]['client_age'].mean())

Выведем сводную таблицу по типу занятости и возраста клиента. Убедимся, что нулевых значений в возрасте больше нет.

In [18]:
df.pivot_table(index='employment_type', values='client_age', aggfunc=['count', 'min', 'max', 'mean'])

,count,min,max,mean
,client_age,client_age,client_age,client_age
employment_type,,,,
безработный,2,31,45,38.000000
в декрете,1,39,39,39.000000
госслужащий,1459,19,75,40.801234
компаньон,5085,19,74,39.850934
пенсионер,3856,22,74,59.369035
предприниматель,2,27,58,42.500000
сотрудник,11119,19,74,40.018887
студент,1,22,22,22.000000


Чтобы исправить значения в столбце `days_employed` по безработным и пенсионерам - я предлагаю разбить их на группы по квантилям и далее соотнести среднее значение в столбце относительно их группе по возрасту.  
Выведем сводную таблицу по возрасту клиентов.

In [19]:
df['client_age'].quantile([0.25, 0.50, 0.75])

0.25    34.0
0.50    43.0
0.75    53.0
Name: client_age, dtype: float64

На основе таблицы можно созать 4 группы: 
* до 34
* 34-43
* 43-53
* от 53

Создадим новый столбец `age_group` и соотнесем возраст клентов относительно этих групп.

In [20]:
df['age_group'] = pd.qcut(df['client_age'], q=4, labels=['до 34', '34-43', '43-53', 'от 53'])

Чтобы убедиться, что все верно - выведем таблицу со средними значениями возраста по группам.

In [21]:
df.groupby('age_group')['client_age'].mean()

age_group
до 34    28.795215
34-43    38.985247
43-53    48.404444
от 53    59.963095
Name: client_age, dtype: float64

Видно, что возраста соответствуют нашим группам.  
Выведем сводную таблицу со средним значением трудового стажа по возрастным категориям без учета пенсионеров и безработных.

In [22]:
df[(df['employment_type']!='пенсионер')&(df['employment_type']!='безработный')].pivot_table(index='age_group', 
                                                                                            values='days_employed', 
                                                                                            aggfunc=['mean'])

,mean
,days_employed
age_group,
до 34,1480.996369
34-43,2367.381124
43-53,2992.900459
от 53,3494.225977


Теперь предлагаю заменить значения для пенсионеров и безработных в столбце `days_employed` относительно их возрастной группе на среднее которые мы получили выше.  
Так же заменим пустые значения трудового стажа на средние по возрастной категории.

In [23]:
# сделаем датасет без учета пенсионеров и безработных чтобы получить из него средние значения
clear_df = df[(df['employment_type']!='пенсионер')&(df['employment_type']!='безработный')]

# создадим список возрастных групп
age_group = df['age_group'].unique()
client_type

# пройдемся по списку возрастных групп и заменим значения у пенсионеров и безработных в основном датасете на средние значения из датасета без них
for group in age_group:
    df.loc[(df['employment_type']=='пенсионер')&(df['age_group']==group),'days_employed'] = clear_df[clear_df['age_group']== group]['days_employed'].mean()
    df.loc[(df['employment_type']=='безработный')&(df['age_group']==group),'days_employed'] = clear_df[clear_df['age_group']== group]['days_employed'].mean()
    df.loc[(df['days_employed'].isna())&(df['age_group']==group),'days_employed'] = clear_df[clear_df['age_group']== group]['days_employed'].mean()

Убедимся, что пустые значения в столбце `days_employed` заменились, далее выведем сводную таблицу и посмотрим на полученные значения.

In [24]:
df.isna().sum()

number_of_children       0
days_employed            0
client_age               0
education                0
education_id             0
family_status            0
family_status_id         0
gender                   0
employment_type          0
credit_debt              0
monthly_income        2174
credit_purpose           0
age_group                0
dtype: int64

In [25]:
df.pivot_table(index='employment_type', values='days_employed', aggfunc='mean')

,days_employed
employment_type,
безработный,2236.948414
в декрете,3296.759962
госслужащий,3300.417500
компаньон,2137.371356
пенсионер,3404.826202
предприниматель,2007.537030
сотрудник,2330.761214
студент,578.751554


In [26]:
df.pivot_table(index='employment_type', values='client_age', aggfunc=['min', 'max', 'mean'])

,min,max,mean
,client_age,client_age,client_age
employment_type,,,
безработный,31,45,38.000000
в декрете,39,39,39.000000
госслужащий,19,75,40.801234
компаньон,19,74,39.850934
пенсионер,22,74,59.369035
предприниматель,27,58,42.500000
сотрудник,19,74,40.018887
студент,22,22,22.000000


Мы получили трудовой стаж по дням уже более приближенный к реальным значениям, но категория "пенсионеры" все равно вызывает сомнения, видимо они получили такое маленькое значение из-за большого разброса по возрасту внутри категории, есть "пенсионеры" с возрастом 22 года.

Теперь займемся столбцом `monthly_income`. Посмотрим на медианное значения того, сколько ежемесячно получают в разных категориях.

In [27]:
df.pivot_table(index='employment_type', values='monthly_income', 
               aggfunc='median').sort_values('monthly_income', ascending=False)

,monthly_income
employment_type,
предприниматель,499163.144947
компаньон,172357.950966
госслужащий,150447.935283
сотрудник,142594.396847
безработный,131339.751676
пенсионер,118514.486412
студент,98201.625314
в декрете,53829.130729


Из таблицы видно, что предприниматели получают сильно больше остальных. По остальным категориям не сильный разброс. Скорее всего данные зарплаты представлены для крупного города РФ.

Посмотрим у скольких человек и в каких группах пропущенны сведения о доходе.

In [28]:
df[df['monthly_income'].isna()].groupby('employment_type')['client_age'].count()

employment_type
госслужащий         147
компаньон           508
пенсионер           413
предприниматель       1
сотрудник          1105
Name: client_age, dtype: int64

Заменим пропущенные значения в столбце `monthly_income` на медианные значения по группе.

In [29]:
type_employment = df['employment_type'].unique()

for employment in type_employment:
    df.loc[(df['monthly_income'].isna())&(df['employment_type']==employment), 'monthly_income'] = df.loc[df['employment_type']==employment]['monthly_income'].median()

Посмотрим на получившийся результат и убедимся, что больше нет пропущенных значений.

In [30]:
df.pivot_table(index='employment_type', values='monthly_income', 
               aggfunc='median').sort_values('monthly_income', ascending=False)

,monthly_income
employment_type,
предприниматель,499163.144947
компаньон,172357.950966
госслужащий,150447.935283
сотрудник,142594.396847
безработный,131339.751676
пенсионер,118514.486412
студент,98201.625314
в декрете,53829.130729


In [31]:
df.isna().sum()

number_of_children    0
days_employed         0
client_age            0
education             0
education_id          0
family_status         0
family_status_id      0
gender                0
employment_type       0
credit_debt           0
monthly_income        0
credit_purpose        0
age_group             0
dtype: int64

### Обработка столбцов

Начнем со стобца `number_of_children`. Выведем информацию по всем значениям и их количеству.

In [32]:
df.groupby('number_of_children')['number_of_children'].count().sort_values(ascending=False)

number_of_children
 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: number_of_children, dtype: int64

Можно сделать вывод, что количество детей в семье варьируется от 0 до 5. Значения -1 и 20 больше похожи на опечатку, поэтому предлагаю отрицательное значение заменить на положительное, а второе заменить на 2.

In [33]:
df['number_of_children'] = df['number_of_children'].abs()

In [34]:
df.loc[df['number_of_children']==20,'number_of_children'] = 2

Убедимся, что все значения исправились.

In [35]:
df.groupby('number_of_children')['number_of_children'].count().sort_values(ascending=False)

number_of_children
0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: number_of_children, dtype: int64

Теперь займемся столбцом `education`.
Выведем уникальные значения внутри столбца.

In [36]:
df['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

Как мы видим, некоторые значения совпадают, но записаны в разном регистре, приведем их к единому виду и выведем новый список уникальных значений.

In [37]:
df['education'] = df['education'].str.lower()

In [38]:
df['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

Посмотрим на значения в других столбцах, может заметить еще что-нибудь.

In [39]:
df['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [40]:
df['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

В столбце `gender` есть непонятное значение 'XNA'. Предлагаю посмотреть сколько таикх значений содержится в столбце.

In [41]:
df.groupby('gender')['gender'].count()

gender
F      14236
M       7288
XNA        1
Name: gender, dtype: int64

Значение всего одно, поэтому предлагаю его удалить, потому что достоверно восстановить его не получится.

In [42]:
df = df[df['gender']!='XNA']

In [43]:
df.groupby('gender')['gender'].count()

gender
F    14236
M     7288
Name: gender, dtype: int64

**Вывод**  
1) Мы заменили значения столбцов на более интуетивно понятные.  
2) В столбце с трудовым стажем сделали положительными значения которые были отрицательными. Пропущенные значения и экстримально большие заменили на среднее значения по возрастной группе, но полученый результат все равно оказался далек от действительности. Скорее всего была допущена ошибка при выгрузке данных.  
3) Пропущенные значения в столбце с ежемесячным доходом заменили на медианное значение по типу занятости. На мой, взгляд получились правдоподобные результаты для большого города, но категория "безработные" вызывает вопросы, так как в ней большой ежемесячный доход, возможно работают неофициально.  
4) В столбце с количеством детей заменили отрицательные значения на положительные, значени "20" заменили на "2" так как очень похоже на опечатку.  
5) В столбце с образованием привели все к одному регистру.  
6) В столбце с полом было 1 непонятное значение, которое мы так же заменили.  

Теперь можно занятся изменением типа данных и удалением дубликатов.

### Замена типа данных

Выведем информацию о датасете

In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21524 entries, 0 to 21524
Data columns (total 13 columns):
number_of_children    21524 non-null int64
days_employed         21524 non-null float64
client_age            21524 non-null int64
education             21524 non-null object
education_id          21524 non-null int64
family_status         21524 non-null object
family_status_id      21524 non-null int64
gender                21524 non-null object
employment_type       21524 non-null object
credit_debt           21524 non-null int64
monthly_income        21524 non-null float64
credit_purpose        21524 non-null object
age_group             21524 non-null category
dtypes: category(1), float64(2), int64(5), object(5)
memory usage: 2.2+ MB


Предлагаю заменить тип данных у столбца `days_employed` с float64 на int, ведь считаем целыми днями.  
Так же поменяем тип данных у столбца `monthly_income`, излишняя точность здесь не нужна.  
Воспользуемся методом **astype** из библиотеки **pandas** так как мы заменили все пропущенные значения.

In [45]:
df['days_employed'] = df['days_employed'].astype('int')
df['monthly_income'] = df['monthly_income'].astype('int')

In [46]:
print(f"Тип данных у столбца 'days_employed' - {df['days_employed'].dtypes}")
print(f"Тип данных у столбца 'monthly_income' - {df['monthly_income'].dtypes}")

Тип данных у столбца 'days_employed' - int64
Тип данных у столбца 'monthly_income' - int64


**Вывод**  
Мы изменили тип данных у двух столбцов: `days_employed` и `monthly_income` на целочисленный для удобства подсчета дней и ежемесячной зарплаты.

### Обработка дубликатов

Для поиска дубликатов воспользуемся методом **duplicated**, с помощью этого метода выведем количество дубликатов и посмотрим на них

In [47]:
print(f'Количество дубликатов - {df.duplicated().sum()}')

Количество дубликатов - 71


In [48]:
df[df.duplicated()].sort_values(by=['monthly_income', 'days_employed'])

,number_of_children,days_employed,client_age,education,education_id,family_status,family_status_id,gender,employment_type,credit_debt,monthly_income,credit_purpose,age_group
3290,0,3494,58,среднее,1,гражданский брак,1,F,пенсионер,0,118514,сыграть свадьбу,от 53
4851,0,3494,60,среднее,1,гражданский брак,1,F,пенсионер,0,118514,свадьба,от 53
5557,0,3494,58,среднее,1,гражданский брак,1,F,пенсионер,0,118514,сыграть свадьбу,от 53
7808,0,3494,57,среднее,1,гражданский брак,1,F,пенсионер,0,118514,на проведение свадьбы,от 53
7921,0,3494,64,высшее,0,гражданский брак,1,F,пенсионер,0,118514,на проведение свадьбы,от 53
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17774,1,2367,40,среднее,1,гражданский брак,1,F,компаньон,0,172357,строительство жилой недвижимости,34-43
19387,0,2367,38,высшее,0,гражданский брак,1,F,компаньон,0,172357,на проведение свадьбы,34-43
15991,0,2992,51,среднее,1,гражданский брак,1,F,компаньон,0,172357,на проведение свадьбы,43-53
19369,0,2992,45,среднее,1,гражданский брак,1,F,компаньон,0,172357,свадьба,43-53


Из таблицы видно, что некоторые значения действительно очень похожи. Возможно клиент несколько раз подавал заявку, поэтому появились задвоения. Предлагаю их удалить и убедиться что их больше нет.  
Для удаления дубликтов воспользуемся стандартным методом **drop_duplicates**.

In [49]:
df = df.drop_duplicates()
print(f'Количество дубликатов - {df.duplicated().sum()}')

Количество дубликатов - 0


**Вывод**  
Мы удалили дубликаты из датафрейма так как они очень похожи на задвоения. Могли появиться из-за того, что клиент несколько раз подавал заявку.

### Лемматизация

Проведем лемматизацию и посмотрим на что чаще берут кредит.

In [50]:
# пройдемся циклом по столбцу 'credit_purpose' и лемматезируем из него значения, результат сохраним в пустой список purpose 
m = Mystem()
purpose = []
for text in df['credit_purpose']:
    lemmas = m.lemmatize(text)
    purpose.extend(lemmas)

Создадим датафрейм на из полученного списка и отсортируем его по убыванию.

In [51]:
index = []
count = []
for i,k in Counter(purpose).items():
    index.append(i)
    count.append(k)
df_purpose = pd.Series(index=index, data=count)

In [52]:
df_purpose.sort_values(ascending=False)

                  33569
\n                21453
недвижимость       6350
покупка            5896
жилье              4460
автомобиль         4306
образование        4013
с                  2918
операция           2604
свадьба            2324
свой               2230
на                 2222
строительство      1878
высокий            1374
получение          1314
коммерческий       1311
для                1289
жилой              1230
сделка              941
дополнительный      906
заниматься          904
проведение          768
сыграть             765
сдача               651
семья               638
собственный         635
со                  627
ремонт              607
подержанный         486
подержать           478
приобретение        461
профильный          436
dtype: int64

По результатм лемматизации можно выделить следующие категории для взятия кредита:
* недвижимость и жилье можно объеденить в одну группу (недвижимость)
* автомобиль
* образование
* операция
* свадьба

Создадим отдельный столбец с этими категориями в основном датафрейме чтобы была понятней цель кредита. 

In [53]:
# создаем список категорий
categories = ['недвижимость', 'жилье', 'автомобиль', 'образование', 'операция', 'свадьба']

# проведем лемматизацию по строкам, если в строке есть значение из нашего списка категорий, то мы добавляем в новый столбец эту категорию

def lemmatize(text):
    lemma = m.lemmatize(text)
    for word in categories:
        if word in lemma:
            if (word == 'недвижимость') or (word == 'жилье'):
                return 'Недвижимость'
            elif word == 'автомобиль':
                return 'Автомобиль'
            elif word == 'образование':
                return 'Образование'
            elif word == 'операция':
                return 'Операция'
            elif word == 'свадьба':
                return 'Свадьба'
    return lemma

df['purpose_group'] = df['credit_purpose'].apply(lemmatize)  

Выведем датафрейм и посмотрим на получившиемся результаты. Так же выведем количество получившихся категорий.

In [54]:
df.head(10)

,number_of_children,days_employed,client_age,education,education_id,family_status,family_status_id,gender,employment_type,credit_debt,monthly_income,credit_purpose,age_group,purpose_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,34-43,Недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,34-43,Автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,до 34,Недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,до 34,Образование
4,0,2992,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,43-53,Свадьба
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,до 34,Недвижимость
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,34-43,Недвижимость
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,43-53,Образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,34-43,Свадьба
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,34-43,Недвижимость


In [55]:
df['purpose_group'].value_counts()

Недвижимость    10810
Автомобиль       4306
Образование      4013
Свадьба          2324
Name: purpose_group, dtype: int64

**Вывод**  
После проведения лемматизации у нас получилось 4 основных цели для взятия кредита:
* Покупка недвижимости
* Покупка автомобиля
* На образование
* На свадьбу

Из полученных данных можно сказать, что на покупку недвижимости люди чаще берут кредты.

### Категоризация данных

Ранее мы уже созвавали два столбца с катеригориями, один по возрасту `age_group`, второй по цели кредита `purpose_group`. Необходимо создать еще одну категорию по доходам, для этого предлагаю воспользоваться квантилями. Я думаю, так будет правильней и понятней для разбивки дохода на группы. Выведем квантили и далее разобьем столбец с доходом на эти категории.

In [56]:
df['monthly_income'].quantile([0.25, 0.50, 0.75])

0.25    107620.0
0.50    142594.0
0.75    195818.0
Name: monthly_income, dtype: float64

In [57]:
df['income_group'] = pd.qcut(df['monthly_income'], q=4, labels=['маленький доход', 'средний доход', 'большой доход', 'очень большой доход'])

In [58]:
df.pivot_table(index='income_group', values='monthly_income', aggfunc=['count', 'min', 'max', 'median'])

,count,min,max,median
,monthly_income,monthly_income,monthly_income,monthly_income
income_group,,,,
маленький доход,5364,20667,107620,83881
средний доход,5479,107632,142594,128089
большой доход,5247,142595,195818,167170
очень большой доход,5363,195821,2265604,254303


Посмотрим на информацию о человеке с самым большим заработком.

In [59]:
df[df['monthly_income']==2265604]

,number_of_children,days_employed,client_age,education,education_id,family_status,family_status_id,gender,employment_type,credit_debt,monthly_income,credit_purpose,age_group,purpose_group,income_group
12412,0,1477,44,высшее,0,женат / замужем,0,M,компаньон,0,2265604,ремонт жилью,43-53,Недвижимость,очень большой доход


Так же создадим столбец с категориями о том есть у клиента дети или нет. И посмотрим на получившийся результат.

In [60]:
def client_children(count):
    if count > 0:
        return 'Есть дети'
    else:
        return 'Нет детей'

df['having_children'] = df['number_of_children'].apply(client_children)

In [61]:
df.head()

,number_of_children,days_employed,client_age,education,education_id,family_status,family_status_id,gender,employment_type,credit_debt,monthly_income,credit_purpose,age_group,purpose_group,income_group,having_children
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,34-43,Недвижимость,очень большой доход,Есть дети
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,34-43,Автомобиль,средний доход,Есть дети
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,до 34,Недвижимость,большой доход,Нет детей
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,до 34,Образование,очень большой доход,Есть дети
4,0,2992,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,43-53,Свадьба,большой доход,Нет детей


**Вывод**  

Мы сделали еще один столбец `income_group` для разбивки клиентов по категориям доходов. Для разбивки использовали квантили и получили 4 группы:
* маленький доход	
* средний доход
* большой доход
* очень большой доход 

Из полученных значений видно, что максимальные значения в группах соответсвтуют нашим квантилям.  
В группе с очень большим доходом есть человек с доходом который в разы отличается от медианного значения по этой группе, возможно это выброс.  

Так же мы создали столбец для разбивки клиентов по наличию детей.

## Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [62]:
df.pivot_table(index='having_children', values='credit_debt', aggfunc=['count', 'sum', 'mean'])

,count,sum,mean
,credit_debt,credit_debt,credit_debt
having_children,,,
Есть дети,7363,678,0.092082
Нет детей,14090,1063,0.075444


**Вывод**  
 
Из полученых результатов мы видим, что количество людей без детей составляет 14091 человек, из которых у 7.5% есть просрочка по кредиту. Люди с детьми почти в 2 раза реже берут кредиты, но при этом доля людей с просрочкой по кредиту составляет уже 9.2%.
При соотношении резульатов, можно сделать вывод, что люди с детьми реже возращают кредит в срок.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [63]:
df.pivot_table(index='family_status', values='credit_debt', aggfunc=['count', 'sum', 'mean']).sort_values(('mean','credit_debt'), ascending=False)

,count,sum,mean
,credit_debt,credit_debt,credit_debt
family_status,,,
Не женат / не замужем,2810,274,0.097509
гражданский брак,4150,388,0.093494
женат / замужем,12339,931,0.075452
в разводе,1195,85,0.071130
вдовец / вдова,959,63,0.065693


**Вывод**  

Из таблицы видно, что люди относящиеся с категории "женат/замужем" в несколько раз чаще берут кредиты, у нас их 12339 человек. Скорее всего это связано с покупкой квартиры для расширения жилой площади, но при этом доля просроченных кредитов у них составляет 7.5%, что выше чем у категории "в разводе" с 1195 людьми и просрочкой 7.1% и выше чем у категории "вдовец/вдова" с 959 людьми и просрочкой 6.6%. 
Так же стоит обратить внимание, что категория "Не женат/не замужем" состоящая из 2810 человек имеет самый высокий процент по просрочке - 9.8%, скорее всего это связано с тем, что человеку без пары сложнее выплачивать кредит, но при этом есть категория "гражданский брак" в которой 4151 и просрочка в ней составляет 9.3%, что почти на 2% выше чем у категории "женат/замужем".  
Можно сделать вывод, что одинокие люди реже берут кредит и у них чаще бывает просрочка по кредиты, но если человек ранее был "женат/замужем", то процент просрочки снижается, видимо это связано, что осталось наследство от второй половинки.  
Люди имеющие второю половинку намного чаще берут кредиты и люди находящиеся в официальном браке намного чаще платят в срок, видимо это говорит о серьезности пары.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [64]:
df.pivot_table(index='income_group', values='credit_debt', aggfunc=['count', 'sum', 'mean']).sort_values(('mean','credit_debt'), ascending=False)

,count,sum,mean
,credit_debt,credit_debt,credit_debt
income_group,,,
средний доход,5479,483,0.088155
большой доход,5247,448,0.085382
маленький доход,5364,427,0.079605
очень большой доход,5363,383,0.071415


**Вывод**  

Из таблицы видно, что в каждой категории примерно одинаковое количество человек которое берет кредиты - чуть выше 5000 человек.  
На удивление, люди с маленьким доходом платят лучше чем люди с большим и средним доходом, доля просрочки людей с маленьким доходом составляет 8%, по большим 8.5%, а по средним 8.8%.  
Люди с очень большим доходом платят лучше всех, у них доля просрочки составляет всего 7.1%.  
Я думаю, что люди с меньшим доходом привыкли контролировать свой бюджет, поэтому они страются закрывать все вовремя.

- Как разные цели кредита влияют на его возврат в срок?

In [65]:
df.pivot_table(index='purpose_group', values='credit_debt', aggfunc=['count', 'sum', 'mean']).sort_values(('mean','credit_debt'), ascending=False)

,count,sum,mean
,credit_debt,credit_debt,credit_debt
purpose_group,,,
Автомобиль,4306,403,0.093590
Образование,4013,370,0.092200
Свадьба,2324,186,0.080034
Недвижимость,10810,782,0.072340


**Вывод**  

Из таблицы видно, что самая популярная категория это "недвижимость", к ней относятся 10811 человек. В это группе самый низкий процент просрочки по кредитам 7.2%, видимо люди ответственно подходят к покупке недвижимости.
Категория "автомобиль" является самой просрачиваемой, в ней 4306 человек и доля просроченных кредитов составляет 9.4%. Видимо покупка автомобиля несет за собой дополнительные финансовые растраты: заправка, починка и т.д.
Так же категория "образование" имеет достаточно высокий процент просроченных кредитов, в этой категории 4013 и доля просрочки составляет 9.2%. Скорее всего, это связано с тем, что люди не могут совмещать работу с учебой и получать достаточно денег для погашения кредита.  
К категории "свадьба" относятся 2324 и доля просроченных кредитов в ней 8%, это ниже более чем на 1% чем у категорий "автомобиль" и "образование", хотя количество человек в почти в 2 раза меньше.

## Общий вывод

Из полученных данных можно составить портрет идеального клиента для банка - это человек находящйся в браке, без детей и с очень высоким уровнем дохода. Скорее всего это связано с тем, что у человека есть вторая половинка, которая может помогать финансово, нет лишних затрат на содержание ребенка и при этом у него высокий уровень дохода что дает некую гарантию.